In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
from collections import defaultdict
from gensim import corpora

documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

n_docs = len(documents)

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

dictionary = corpora.Dictionary(texts)
n_words = len(dictionary)
corpus = [dictionary.doc2bow(text) for text in texts]
print("n_docs", n_docs)
print('n_words', n_words)
corpus

2021-03-22 14:01:22,580 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-03-22 14:01:22,580 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


n_docs 9
n_words 12


[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(2, 1), (5, 1), (7, 1), (8, 1)],
 [(1, 1), (5, 2), (8, 1)],
 [(3, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(4, 1), (10, 1), (11, 1)]]

In [3]:
from gensim import models

lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)

2021-03-22 14:15:40,316 : INFO : using serial LSI version on this node
2021-03-22 14:15:40,321 : INFO : updating model with new documents
2021-03-22 14:15:40,323 : INFO : preparing a new chunk of documents
2021-03-22 14:15:40,336 : INFO : using 100 extra samples and 2 power iterations
2021-03-22 14:15:40,336 : INFO : 1st phase: constructing (12, 102) action matrix
2021-03-22 14:15:40,343 : INFO : orthonormalizing (12, 102) action matrix
2021-03-22 14:15:40,350 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2021-03-22 14:15:40,356 : INFO : computing the final decomposition
2021-03-22 14:15:40,357 : INFO : keeping 2 factors (discarding 43.156% of energy spectrum)
2021-03-22 14:15:40,358 : INFO : processed documents up to #9
2021-03-22 14:15:40,358 : INFO : topic #0(3.341): 0.644*"system" + 0.404*"user" + 0.301*"eps" + 0.265*"time" + 0.265*"response" + 0.240*"computer" + 0.221*"human" + 0.206*"survey" + 0.198*"interface" + 0.036*"graph"
2021-03-22 14:15:40,359 : INFO : topic #1(2

In [4]:
doc = 'Human computer interaction'
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]  # convert the query to LSI space
print(vec_bow)
print(vec_lsi)

[(0, 1), (1, 1)]
[(0, 0.46182100453271546), (1, -0.0700276652790009)]


In [12]:
from gensim import similarities

index = similarities.MatrixSimilarity(lsi[corpus])  # transform corpus to LSI space and index it

print(len(index))
for doc in index:
    print(doc)  # len(doc) = 9

index.save('/tmp/deerwester.index')
index = similarities.MatrixSimilarity.load('/tmp/deerwester.index')

2021-03-22 14:37:27,337 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2021-03-22 14:37:27,343 : INFO : creating matrix with 9 documents and 2 features
2021-03-22 14:37:27,351 : INFO : saving MatrixSimilarity object under /tmp/deerwester.index, separately None
2021-03-22 14:37:27,380 : INFO : saved /tmp/deerwester.index
2021-03-22 14:37:27,380 : INFO : loading MatrixSimilarity object from /tmp/deerwester.index
2021-03-22 14:37:27,396 : INFO : loaded /tmp/deerwester.index


9
[ 1.          0.9142159   0.999982    0.99478287  0.87990767 -0.18518142
 -0.16756734 -0.16003224 -0.01170431]
[0.9142159  1.         0.9166299  0.8681071  0.9969673  0.228923
 0.2463051  0.2537008  0.39449945]
[ 0.999982    0.9166299   1.          0.99415314  0.8827417  -0.17928416
 -0.16165146 -0.154109   -0.00570685]
[ 0.99478287  0.8681071   0.99415314  1.          0.8268452  -0.28446588
 -0.26726568 -0.25989753 -0.11365126]
[0.87990767 0.9969673  0.8827417  0.8268452  1.         0.30398417
 0.32098264 0.3282073  0.4648134 ]
[-0.18518142  0.228923   -0.17928416 -0.28446588  0.30398417  0.99999994
  0.99983984  0.999674    0.98480445]
[-0.16756734  0.2463051  -0.16165146 -0.26726568  0.32098264  0.99983984
  1.          0.9999708   0.9877544 ]
[-0.16003224  0.2537008  -0.154109   -0.25989753  0.3282073   0.999674
  0.9999708   1.          0.98891723]
[-0.01170431  0.39449945 -0.00570685 -0.11365126  0.4648134   0.98480445
  0.9877544   0.98891723  1.        ]


In [16]:
sims = index[vec_lsi]
print(list(enumerate(sims)))

print("")
print("Similarities with 'Human computer interaction':")
# sorted similarities
sims = sorted(enumerate(sims), key=lambda item: -item[1])
for doc_position, doc_score in sims:
    print(doc_score, documents[doc_position])

[(0, 0.998093), (1, 0.93748635), (2, 0.9984453), (3, 0.9865886), (4, 0.90755945), (5, -0.12416792), (6, -0.10639259), (7, -0.09879464), (8, 0.050041765)]

Similarities with 'Human computer interaction'
0.9984453 The EPS user interface management system
0.998093 Human machine interface for lab abc computer applications
0.9865886 System and human system engineering testing of EPS
0.93748635 A survey of user opinion of computer system response time
0.90755945 Relation of user perceived response time to error measurement
0.050041765 Graph minors A survey
-0.09879464 Graph minors IV Widths of trees and well quasi ordering
-0.10639259 The intersection graph of paths in trees
-0.12416792 The generation of random binary unordered trees
